In [1]:
from numba.experimental import jitclass
from numba import njit
import numpy as np

%load_ext autoreload
%autoreload 2


In [5]:
class Leverage:
    def __init__(self) -> None:
        pass

    def market_fee_pct(self):
        return 0.0009

    def mmr_pct(self):
        return 0.005

    def max_leverage(self):
        return 150.0

    def leverage_calculator(self):
        raise Exception()

    def calc_liq_price(
        self,
        leverage: float,
        entry_size_usd: float,
        average_entry: float,
        og_available_balance: float,
        og_cash_used: float,
        og_cash_borrowed: float,
    ):
        # Getting Order Cost
        # https://www.bybithelp.com/HelpCenterKnowledge/bybitHC_Article?id=000001064&language=en_US
        initial_margin = entry_size_usd / leverage
        fee_to_open = entry_size_usd * 0.0009  # math checked
        possible_bankruptcy_fee = entry_size_usd * (leverage - 1) / leverage * self.mmr_pct()
        cash_used = initial_margin + fee_to_open + possible_bankruptcy_fee  # math checked

        if cash_used > og_available_balance:
            raise Exception(
                msg=f"Cash used={cash_used} > available_balance={og_available_balance}",
                order_status=1,
            )
        else:
            # liq formula
            # https://www.bybithelp.com/HelpCenterKnowledge/bybitHC_Article?id=000001067&language=en_US
            available_balance = round(og_available_balance - cash_used, 4)
            cash_used = round(og_cash_used + cash_used, 4)
            cash_borrowed = round(og_cash_borrowed + entry_size_usd - cash_used, 4)

            liq_price = average_entry * (1 - (1 / leverage) + self.mmr_pct())  # math checked
            liq_price = liq_price
            can_move_sl_to_be = True

        return (
            leverage,
            liq_price,
            available_balance,
            cash_used,
            cash_borrowed,
            can_move_sl_to_be,
        )


# @jitclass()
class SetStaticLeverage(Leverage):
    def leverage_calculator(
        self,
        static_leverage: float,
        average_entry: float,
        entry_size_usd: float,
        cash_used: float,
        available_balance: float,
        cash_borrowed: float,
        sl_price: float,
    ):
        liq_price = self.calc_liq_price(
            leverage=static_leverage,
            entry_size_usd=entry_size_usd,
            average_entry=average_entry,
            og_cash_used=cash_used,
            og_available_balance=available_balance,
            og_cash_borrowed=cash_borrowed,
        )
        return liq_price


# @jitclass()
class CalcDynamicLeverage(Leverage):
    def leverage_calculator(
        self,
        static_leverage: float,
        average_entry: float,
        entry_size_usd: float,
        cash_used: float,
        available_balance: float,
        cash_borrowed: float,
        sl_price: float,
    ):
        leverage = -average_entry / ((sl_price - sl_price * 0.001) - average_entry - self.mmr_pct() * average_entry)

        if leverage > self.max_leverage():
            # print(f"Setting leverage from {leverage} to max leverage {self.max_leverage()}")
            leverage = self.max_leverage()
        elif leverage < 1:
            # print(f"Setting leverage from {leverage} to {1}")
            leverage = 1
        else:
            # print(f"Leverage set too {leverage}")
            leverage = leverage

        liq_price = self.calc_liq_price(
            leverage=leverage,
            entry_size_usd=entry_size_usd,
            average_entry=average_entry,
            og_cash_used=cash_used,
            og_available_balance=available_balance,
            og_cash_borrowed=cash_borrowed,
        )
        return liq_price


# @njit
def apply(lev: Leverage):
    for i in range(100):
        for a in range(200):
            for bar in range(1000):
                leverage_resuls = lev.leverage_calculator(
                    static_leverage=20.0,
                    average_entry=26557.6,
                    entry_size_usd=26.5513,
                    cash_used=10.0,
                    available_balance=6948.548,
                    cash_borrowed=643.15,
                    sl_price=26544.2,
                )


leverage_calculator = (SetStaticLeverage(), CalcDynamicLeverage())

# print(apply(leverage_calculator[0]))
apply(leverage_calculator[1])